## [03_Big_Data.ipynb](https://github.com/raybellwaves/xskillscore-tutorial/blob/master/03_Big_Data.ipynb)

In this notebook I verify 12 million forecasts in a couple of seconds using the RMSE metric on a `dask.array`.

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import xskillscore as xs
import dask.array as da
from dask.distributed import Client

By default the [`dask.distributed.Client`](https://distributed.dask.org/en/latest/client.html) uses a [`LocalCluster`](https://docs.dask.org/en/latest/setup/single-distributed.html#localcluster)

```
cluster = LocalCluster()
client = Client(cluster)
```

However, this code can easily be adapted to scale on massive datasets using distributed computing via various methods of deployment:

 - [Kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html)
 - [High Performance Computers](https://docs.dask.org/en/latest/setup/hpc.html)
 - [YARN](https://yarn.dask.org/en/latest/)
 - [AWS Fargate](https://aws.amazon.com/fargate/)
 
or vendor products:
 
  - [SaturnCloud](https://www.saturncloud.io/s/)
  
If anyone does run this example on a large cluster I would be curious how big you can scale `nstores` and `nskus` and how long it takes to run `rmse`. You are welcome to post it in the [issue section](https://github.com/raybellwaves/xskillscore-tutorial/issues) with the title **Cluster Benchmark: N forecasts, T time** and include the setup in the description.

Setup the client (i.e. connect to the scheduler):

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:33685 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 16.73 GB


Due to the success of your previous forecast (and verification using xskillscore!) the company you work for has expanded. They have grown to 4,000 stores each with 3,000 products:

In [5]:
nstores = 4000
nskus = 3000

nforecasts = nstores * nskus
print(f"That's {nforecasts:,d} different forecasts to verify!")

stores = np.arange(nstores)
skus = np.arange(nskus)

That's 12,000,000 different forecasts to verify!


The time period of interest is the same dates but for 2021:

In [6]:
dates = pd.date_range("1/1/2021", "1/5/2021", freq="D")

Setup the data as a `dask.array` of dates x stores x skus.

`dask` uses similar functions as `numpy`. In this case switch the `np.` to `da.` to generate random numbers between 1 and 10:

In [7]:
data = da.random.randint(9, size=(len(dates), len(stores), len(skus))) + 1
data

dask.array<add, shape=(5, 4000, 3000), dtype=int64, chunksize=(5, 1000, 1500), chunktype=numpy.ndarray>

Put this into an `xarray.DataArray` and specify the Coordinates and dimensions:

In [8]:
y = xr.DataArray(data, coords=[dates, stores, skus], dims=["DATE", "STORE", "SKU"])
y

<xarray.DataArray 'add-d339578a86fb65deb2f294f7ccf4f817' (DATE: 5, STORE: 4000, SKU: 3000)>
dask.array<add, shape=(5, 4000, 3000), dtype=int64, chunksize=(5, 1000, 1500), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-05
  * STORE    (STORE) int64 0 1 2 3 4 5 6 ... 3993 3994 3995 3996 3997 3998 3999
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

Create a prediction array similar to that in [01_Deterministic.ipynb](https://github.com/raybellwaves/xskillscore-tutorial/blob/master/01_Determinisitic.ipynb):

In [9]:
noise = da.random.uniform(-1, 1, size=(len(dates), len(stores), len(skus)))
yhat = y + (y * noise)
yhat

<xarray.DataArray (DATE: 5, STORE: 4000, SKU: 3000)>
dask.array<add, shape=(5, 4000, 3000), dtype=float64, chunksize=(5, 1000, 1500), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-05
  * STORE    (STORE) int64 0 1 2 3 4 5 6 ... 3993 3994 3995 3996 3997 3998 3999
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

Finally caculate RMSE at the store and sku level.

Use the [`.compute()`](https://distributed.dask.org/en/latest/manage-computation.html) method to return the values:

In [14]:
%time xs.rmse(y, yhat, 'DATE').compute()

CPU times: user 157 ms, sys: 63.7 ms, total: 221 ms
Wall time: 1.13 s


<xarray.DataArray (STORE: 4000, SKU: 3000)>
array([[1.42629153, 2.71567017, 3.80683411, ..., 3.28007643, 4.60848796,
        2.86241048],
       [2.26560765, 3.16809256, 3.87118172, ..., 4.10125899, 1.93652809,
        2.62229031],
       [3.14355972, 3.0756653 , 1.30366742, ..., 2.75627498, 4.18024531,
        2.92426584],
       ...,
       [3.7880626 , 3.11271962, 3.33285011, ..., 2.02237044, 2.06700284,
        2.7979034 ],
       [3.16773592, 2.04505072, 4.65989558, ..., 4.48585311, 2.55574262,
        2.9556035 ],
       [2.87047399, 3.46967357, 2.64280147, ..., 2.7631796 , 5.05430802,
        4.18419859]])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3 4 5 6 ... 3993 3994 3995 3996 3997 3998 3999
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999